In [1]:
from functions import g_D_symbolic_coefficients_dict
from functions import restore_matrices
from functions import Augmented_Lagrangian
from functions import update_Lagrangian_coefficients
from functions import jac

In [50]:
D = 3
L = 6
d = 4
rho = 10

In [51]:
expanded_result, coefficients_dict,polynomial,x = g_D_symbolic_coefficients_dict(D)
print("Expanded Result:")
print(expanded_result)
print("\nCoefficients Dictionary:")
for vars_tuple, coeff in coefficients_dict.items():
    print(f"Coefficient of {vars_tuple}: {coeff}")

orders_list = list(coefficients_dict.keys())
coefficients_list = list(coefficients_dict.values())
print(orders_list)
print(coefficients_list)

Expanded Result:
2.66666666666667*x1**4 + x1**3/27 + x1**2*x2/9 + x1**2*x3/9 - 2.66666666666667*x1**2 + x1*x2**2/9 + 2*x1*x2*x3/9 + x1*x3**2/9 + 2.66666666666667*x2**4 + x2**3/27 + x2**2*x3/9 - 2.66666666666667*x2**2 + x2*x3**2/9 + 2.66666666666667*x3**4 + x3**3/27 - 2.66666666666667*x3**2 + 1.0

Coefficients Dictionary:
Coefficient of (4, 0, 0): 2.66666666666667
Coefficient of (3, 0, 0): 1/27
Coefficient of (2, 1, 0): 1/9
Coefficient of (2, 0, 1): 1/9
Coefficient of (2, 0, 0): -2.66666666666667
Coefficient of (1, 2, 0): 1/9
Coefficient of (1, 1, 1): 2/9
Coefficient of (1, 0, 2): 1/9
Coefficient of (0, 4, 0): 2.66666666666667
Coefficient of (0, 3, 0): 1/27
Coefficient of (0, 2, 1): 1/9
Coefficient of (0, 2, 0): -2.66666666666667
Coefficient of (0, 1, 2): 1/9
Coefficient of (0, 0, 4): 2.66666666666667
Coefficient of (0, 0, 3): 1/27
Coefficient of (0, 0, 2): -2.66666666666667
Coefficient of (0, 0, 0): 1.00000000000000
[(4, 0, 0), (3, 0, 0), (2, 1, 0), (2, 0, 1), (2, 0, 0), (1, 2, 0), (1,

In [52]:
orders_list = [(4, 0), (3, 0), (2, 1), (2, 0), (1, 2), (0, 4), (0, 3), (0, 2), (0, 0)]
coefficients_list = [4.00000000000000, 1/8, 3/8, -4.00000000000000, 3/8, 4.00000000000000, 1/8, -4.00000000000000, 1.00000000000000]

In [53]:
from scipy.integrate import quad
# This block is for initialization according to the article, µ, {R_i^l, S_i^l},is initialized randomly using a distribution U ([−1, 1]) uniform over [−1, 1].

# def function of the moments in M_d
def f(x,i,j):
    return 1/2*x**(i+j)

# def function of the function in M_{d-1}
def g(x,i,j):
    return 1/2*(1-x**2)*x**(i+j)

#function generate M_d(mu_{i}^{l})
def M_d(d):
    matrix = np.zeros((d+1, d+1))
    for i in range(d+1):
        for j in range(d+1):
            matrix[i,j],_= quad(f,-1,1,args=(i,j))
    return matrix

# function generate M_d(mu_{i}^{l})
def M_d_1(d):
    matrix = np.zeros((d, d))
    for i in range(d):
        for j in range(d):
            matrix[i,j],_= quad(g,-1,1,args=(i,j))
    return matrix

In [54]:
import numpy as np
x_M_D_L_list = [[] for _ in range(D)]
x_M_D_1_L_list = [[] for _ in range(D)]
x_R_L_list = [[] for _ in range(D)]
x_S_L_list = [[] for _ in range(D)]
for l in range(L):
    for i in range(D):
        x_M_D_L_list[i].append(M_d(d))
        x_M_D_1_L_list[i].append(M_d_1(d))
        x_R_L_list[i].append(np.random.uniform(-1, 1, size=(d+1, d+1)))
        x_S_L_list[i].append(np.random.uniform(-1, 1, size=(d, d)))

x_matrix_list_new = x_M_D_L_list+x_R_L_list+x_M_D_1_L_list+x_S_L_list
x_input = np.array([])
for matrix_index in range(len(x_matrix_list_new)):
    for l in range(L):
        flattened_array = x_matrix_list_new[matrix_index][l].flatten()
        x_input = np.concatenate((x_input, flattened_array))
print(len(x_input))
print(x_input)
Lagrangian_coefficient = 0

1476
[ 1.          0.          0.33333333 ... -0.59275731  0.81313147
  0.67026009]


In [55]:
from functools import partial
aug_lagrangian_partial = partial(Augmented_Lagrangian, d=d, D=D, L=L, orders_list=orders_list,
                                 coefficients_list=coefficients_list,
                                 Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
jac_partial = partial(jac, d=d, D=D, L=L, orders_list=orders_list,
                      coefficients_list=coefficients_list,
                      Lagrangian_coefficient=Lagrangian_coefficient, rho=rho)
print(aug_lagrangian_partial(x_input))

IndexError: tuple index out of range

In [47]:
print(aug_lagrangian_partial.keywords['Lagrangian_coefficient'])

10232.263014612272


In [48]:
from scipy.optimize import minimize


result = minimize(aug_lagrangian_partial, x0=x_input,
                  method='L-BFGS-B',
                  jac=jac_partial,
                  options={
                      'gtol': 1e-2,             # Stopping criterion (relative gradient)
                      'ftol': 1e-5,             # Stopping criterion (absolute value)
                  })

print("-"*40)
print("Minimum value of the Augmented Lagrangian function:", result.fun)
print("Was the optimization successful?", result.success)
print("Number of iterations:", result.nit)
print(result.message)
Lagrangian_coefficient = update_Lagrangian_coefficients(d,D,L,result.x,Lagrangian_coefficient,rho)
print("-"*40)

x_input = result.x
print("Lagrangian_coefficient:",Lagrangian_coefficient)

print("-"*40)
x_M_D_L_list,_,_,_= restore_matrices(x_input,d,D,L)

l_product_list = []
for l in range(L):
    moment_product = 1
    for i in range(D):
        moment_product *= x_M_D_L_list[i][l][0,0]
    l_product_list.append(moment_product)

max_index = l_product_list.index(max(l_product_list))

Denominator_product = 1
for i in range(D):
    Denominator_product *=  x_M_D_L_list[i][max_index][0,0]

x_output = {x[i]: x_M_D_L_list[i][max_index][0,1]/Denominator_product  for i in range(D)}
polynomial_values = polynomial.subs(x_output)

print("The final point: ",tuple(x_output.values()))
print("Objective function value:", polynomial_values)

----------------------------------------
Minimum value of the Augmented Lagrangian function: -10781183588.8505
Was the optimization successful? False
Number of iterations: 1
STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
----------------------------------------
Lagrangian_coefficient: -21001856.49595471
----------------------------------------
The final point:  (0.00027712984977343487, 0.00027852033587166114, 0.0002783965280745673)
Objective function value: 0.999999381677115
